# Предсказание цены на автомобили

Датасет содержит 10 000 записей, созданных с целью прогнозирования цен на автомобили. Каждая строка представляет информацию об автомобиле и его цене.

**Описание столбцов:**

- *Brand*: марка автомобиля.

- *Model*: модель автомобиля.

- *Year*: год выпуска.

- *Engine_Size*: размер двигателя в литрах.

- *Fuel_Type*: тип топлива, используемого автомобилем:
    - *Petrol*: автомобили, работающие на бензине.
    - *Diesel*: автомобили, работающие на дизельном топливе.
    - *Hybrid*: автомобили, использующие как топливо, так и электричество.
    - *Electric*: полностью электрические автомобили.

- *Transmission*: тип трансмиссии:
    - *Manual*: механическая коробка передач.
    - *Automatic*: автоматическая коробка передач.
    - *Semi-Automatic*: полуавтоматическая коробка передач.

- *Mileage*: общее расстояние (в км), которое проехал автомобиль.

- *Doors*: количество дверей в автомобиле.

- *Owner_Count*: количество предыдущих владельцев автомобиля.

- *Price*: цена автомобиля (таргет).

In [71]:
import optuna as opt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing  import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

sns.set_theme()